In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
tqdm.pandas()

In [3]:
keys = ['model_cls', 'task_name', 'checkpoint', 'seed']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'

d  = pd.read_pickle(oj(save_dir, 'd.pkl'))

# r = analyze_utils.load_results_and_cache_autoprompt_json(save_dir, save_file='r.pkl')
# r = pd.read_pickle(oj(save_dir, 'r.pkl'))

# # subselect the data
# r = r[r['n_shots'] == 5]
# r = r[r['num_learned_tokens'] == 6]

# # add more keys
# keys_derived = ['task_collection', 'top_prompt_correctness', 'bleu_top_prompt']
# r['task_collection'] = r.task_name.apply(analyze_utils.task_collection)
# r['top_prompt_correctness'] = r['reciprocal_rank'] == 1
# r['gt_prompt'] = r['task_name'].apply(lambda x: iprompt.data.TASKS[x]['description'])
# r['bleu_top_prompt'] = r.progress_apply(lambda x: sentence_bleu(x['gt_prompt'].split(), x['prefixes'].split()), axis=1) # maybe need to split before calling?

# d = r[keys + keys_extra + keys_out + keys_derived]
# d.to_pickle(oj(save_dir, 'd.pkl'))

# Metrics table

In [10]:
metrics = d.groupby(by=['model_cls', 'task_collection', 'checkpoint']).mean(numeric_only=True).reset_index()
# with pd.option_context('display.float_format', lambda x: '%.1e' % x):
with pd.option_context('display.float_format', lambda x: '%.2g' % x):
    for ckpt in metrics.checkpoint.unique():
        print(ckpt)
        m = (
            metrics[metrics.checkpoint == ckpt]
            .pivot_table(index='model_cls', columns='task_collection', values=['reciprocal_rank', 'top_prompt_correctness', 'bleu_top_prompt'])
            .transpose()
        )
        display(m)

EleutherAI/gpt-j-6B


model_cls                               autoprompt  iprompt   suffix
                       task_collection                              
bleu_top_prompt        ANLI               5.8e-234 2.4e-232 1.1e-232
                       DD                 3.5e-235 8.1e-233 5.6e-233
                       Induction          2.8e-233 5.4e-159 7.5e-233
                       Math               2.8e-234 3.1e-232   3e-232
reciprocal_rank        ANLI                  0.054     0.43    1e-10
                       DD                  0.00034    0.009     0.05
                       Induction             0.099     0.35    0.071
                       Math                   0.12      0.7    0.052
top_prompt_correctness ANLI                  0.054     0.39        0
                       DD                        0        0    0.017
                       Induction             0.092     0.28    0.071
                       Math                   0.12     0.61    0.042

facebook/galactica-6.7b


model_cls                               autoprompt  iprompt   suffix
                       task_collection                              
bleu_top_prompt        ANLI               4.7e-234 1.8e-232 2.9e-232
                       DD                 4.5e-234 2.1e-232 1.6e-232
                       Induction          3.2e-233   7e-233 1.2e-232
                       Math                 8e-235 1.4e-232        0
reciprocal_rank        ANLI                 0.0051     0.58    1e-10
                       DD                    1e-10     0.26    0.026
                       Induction              0.12      0.2    1e-10
                       Math                   0.07     0.47     0.33
top_prompt_correctness ANLI                      0     0.52        0
                       DD                        0     0.16        0
                       Induction              0.11    0.062        0
                       Math                  0.042     0.39     0.33

google/flan-t5-xl


model_cls                               iprompt   suffix
                       task_collection                  
bleu_top_prompt        ANLI            1.5e-158 1.2e-232
                       DD              6.4e-233 5.9e-233
                       Induction        3.8e-81 1.2e-232
                       Math            3.2e-159 1.1e-232
reciprocal_rank        ANLI                0.27    1e-10
                       DD                 0.074    0.034
                       Induction           0.16    0.014
                       Math                0.16    1e-10
top_prompt_correctness ANLI                0.21        0
                       DD                 0.059    0.023
                       Induction          0.099        0
                       Math                 0.1        0

# Prompts table

In [14]:
for ckpt in d.checkpoint.unique():
    print(ckpt)
    prompts = (
        d[d.checkpoint == ckpt]
        .sort_values(by=['task_collection', 'reciprocal_rank'], ascending=False)
        .groupby(by=['model_cls', 'task_name'])
        # .first()
    )
    # with pd.option_context('display.float_format', lambda x: '%.1e' % x):
    with pd.option_context('display.max_rows', None):
        display(prompts[['prefixes']].head())

EleutherAI/gpt-j-6B


,prefixes
0,Create a function named `sum
1,Write a method named `sum
2,Add two or more integers together
3,Given three number you must sum
4,Write a method named digitSum
0,'How do you multiply a
1,'How do you multiply an
2,Print integer factorials only
3,'How do you multiply two
4,How to multiply a number by


google/flan-t5-xl


,prefixes
0,prompt: find the sum of
1,Answer: -32.
2,0 Prompt:
3,Answer: 0; Question
4,What is the sum of the
0,Difference or difference between two numbers
1,Difference or difference between two things
2,How many is the difference between
3,What is the difference between 1
4,Let's calculate the difference


facebook/galactica-6.7b


,prefixes
0,In order to add two or
1,Given two decimal places show your
2,Let's see how this
3,Let's add some examples
4,When you add two whole number
0,When we multiply two even or
1,When we multiply two positive or
2,Given two integers a greater than
3,Given a set of two or
4,Given three consecutive even integers whose


In [15]:
for k, g in prompts:
    print(k)
    display(g['prefixes'].shape)

('autoprompt', 'add_two')


(1756,)

('autoprompt', 'antonyms')


(96,)

('autoprompt', 'cause_and_effect')


...

(8,)

('suffix', 'translation_en-es')


(12,)

('suffix', 'word_in_context')


(4,)

In [13]:
d.columns

Index(['model_cls', 'task_name', 'checkpoint', 'seed', 'n_shots',
       'num_learned_tokens', 'reciprocal_rank', 'prefixes',
       'iprompt_preprefix_str', 'task_collection', 'top_prompt_correctness',
       'bleu_top_prompt'],
      dtype='object')